In [1]:
import pandas as pd
import numpy as np
import sympy
from sympy.solvers.inequalities import reduce_rational_inequalities,reduce_inequalities
from itertools import chain
from collections import defaultdict
from IPython.display import display

from sympy.parsing.sympy_parser import parse_expr
from sympy.parsing.sympy_parser import standard_transformations
from sympy.printing.mathml import print_mathml

from IPython.display import display, Markdown, Latex
import os

import seaborn as sns
import matplotlib.pyplot as plt


### graph library: plotly for making interactive graph
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
def make_symbols_positive(sym):
    return sympy.symbols("{}".format(sym),positive =True)

def make_symbols_function(sym):
    return sympy.symbols("{}".format(sym),cl)

def subs_function(target,list_):
    return sympy.simplify(target.subs(list_))

# Data_loading

In [3]:
Path_ = r"C:\Users\Samsung\jupyter\Graduation_thesis\result\version_4"

In [4]:
file_list = os.listdir(Path_)
file_list_xlsx = [f for f in file_list if f.endswith(".xlsx")]
file_name = [li.replace(".xlsx","") for li in file_list_xlsx]

In [5]:
for fn in file_name:
    path_ = Path_ + "\\{}.xlsx"
    globals() ["{}".format(fn)] = pd.read_excel(path_.format(fn),index_col=0,header=None)
    try:
        globals() ["{}".format(fn)] = globals() ["{}".format(fn)]#.rename(index={"profit_all":"pro_all"})
    except:
        pass

In [6]:
for fn in file_name:
    path_ = Path_ + "\\{}.xlsx"
    globals() ["{}".format(fn)] = pd.read_excel(path_.format(fn),index_col=0,header=None)
    try:
        globals() ["{}".format(fn)] = globals() ["{}".format(fn)]#.rename(index={"profit_all":"pro_all"})
    except:
        pass

In [7]:
p,w,c,s,alpha,beta,delta_d,delta_s,epsilon = make_symbols_positive("p,w,c,s,alpha,beta,delta_d,delta_s,epsilon")

In [8]:
parse_dic = {"p":p,"w":w,"c":c,"s":s
             ,"alpha":alpha,"beta":beta
             ,"delta_d":delta_d,"delta_s":delta_s,"epsilon":epsilon }

In [9]:
for fn in file_name:
    globals() ["{}".format(fn)][1] = globals() ["{}".format(fn)][1].map(lambda x: parse_expr(x.replace("^","**").replace("lambda","lambda_"),parse_dic))

In [10]:
case_  = ["ND","DD","SD","BD","BS"]

In [11]:
index_order = ["w","epsilon","T","p","p_w","q"
               ,"profit_r","profit_s","profit_all"
               ,"h1_all_p","h1_r_p","h1_s_w","h1_s_e","h2_s"]

for ca in case_:
    FB_ = globals ()["{}_FB".format(ca)].rename(columns={1:"FB"})
    WH_ = globals ()["{}_WH".format(ca)].rename(columns={1:"WH"})
    QD_ = globals ()["{}_QD".format(ca)].rename(columns={1:"QD"})
    
    globals ()["{}_all".format(ca)] = pd.concat([FB_,WH_,QD_],join="outer",axis=1).reindex(index=index_order)

In [12]:
def root_cal(value):
    return value**(1/2)

# w_range

In [13]:
def sol_w_same(df_1,target_):
    w_sol = sympy.solve([df_1["QD"][target_]-df_1["FB"][target_]],w)
    return w_sol

In [14]:
def sol_w_other(df_1,equa):
    w_sol = sympy.solve(equa,w)
    return w_sol

In [15]:
def display_solve(df_1,w_sol):
    for row,col in df_1.iterrows():
        print(row)
        try:
            display(col["QD"].subs([(w,w_sol)]).factor())
        except:
            print()
        print()

In [16]:
def display_solve2(df_1,w_sol):
    for row,col in df_1.iterrows():
        print(row)
        try:
            display(col["QD"].subs([(w,w_sol)]).simplify())
        except:
            print()
        print()

# ND's prepare

In [17]:
ND_all

,FB,WH,QD
w,NaN,(alpha + beta*c)/(2*beta),w
epsilon,NaN,NaN,(-alpha - 2*beta*c + 3*beta*w)/(beta*(alpha - ...
T,NaN,(alpha + beta*c)/(2*beta),(alpha + 2*beta*c + beta*w)/(4*beta)
p,(alpha + beta*c)/(2*beta),(3*alpha + beta*c)/(4*beta),(3*alpha + 2*beta*c - beta*w)/(4*beta)
p_w,NaN,(alpha + beta*w)/(2*beta),(-2*alpha*beta*epsilon + alpha + beta*w)/(-2*b...
q,alpha/2 - beta*c/2,alpha/4 - beta*c/4,alpha/4 - beta*c/2 + beta*w/4
profit_r,NaN,(alpha - beta*c)**2/(16*beta),(alpha - beta*w)*(alpha/4 - beta*c/2 + beta*w/...
profit_s,NaN,(alpha - beta*c)**2/(8*beta),(alpha/4 - beta*c/2 + beta*w/4)*(alpha - 2*bet...
profit_all,(alpha - beta*c)**2/(4*beta),3*(alpha - beta*c)**2/(16*beta),(3*alpha**2 - 8*alpha*beta*c + 2*alpha*beta*w ...
h1_all_p,-2*beta,NaN,NaN


In [18]:
df_1 = ND_all

In [26]:
w_sol = sol_w_other(df_1,[df_1["QD"]["profit_r"]])
w_sol[0][0]

alpha/beta

In [27]:
display_solve(df_1,w_sol[0][0])

w


alpha/beta


epsilon


1/beta


T


...

zoo*(-alpha + beta*c)**2


h1_s_e


zoo*(-alpha + beta*c)**4


h2_s


zoo*(-alpha + beta*c)**4

In [24]:
w_sol = sol_w_other(df_1,[df_1["QD"]["profit_s"]-df_1["WH"]["profit_s"]])
w_sol[0][0]

-sqrt(2)*(-alpha + beta*c)/beta + (-alpha + 2*beta*c)/beta

In [25]:
display_solve(df_1,w_sol[0][0])

w


-(-sqrt(2)*alpha + alpha - 2*beta*c + sqrt(2)*beta*c)/beta


epsilon


sqrt(2)*(-4 + 3*sqrt(2))/(2*beta)


T


...

-beta*(-1 + 2*sqrt(2))/(4*(-2 + sqrt(2))**2)


h1_s_e


-beta*(-alpha + beta*c)**2/(8*(-2 + sqrt(2))**2)


h2_s


-beta**2*(-alpha + beta*c)**2/(64*(-2 + sqrt(2))**2)

In [28]:
w_sol = sol_w_other(df_1,[df_1["QD"]["profit_all"]-df_1["FB"]["profit_all"]])
w_sol[0][0]

alpha/beta

In [29]:
display_solve(df_1,w_sol[0][0])

w


alpha/beta


epsilon


1/beta


T


...

zoo*(-alpha + beta*c)**2


h1_s_e


zoo*(-alpha + beta*c)**4


h2_s


zoo*(-alpha + beta*c)**4

# DD

In [30]:
df_1 = DD_all

In [31]:
w_sol = sol_w_other(df_1,[df_1["QD"]["profit_r"]])
w_sol[0][0]

(alpha - beta*delta_d*s)/beta

In [32]:
display_solve(df_1,w_sol[0][0])

w


-(-alpha + beta*delta_d*s)/beta


epsilon


1/beta


T


...

4*zoo*(-alpha + beta*c + beta*delta_d*s)**2


h1_s_e


zoo*(-alpha + beta*c + beta*delta_d*s)**4


h2_s


zoo*(-alpha + beta*c + beta*delta_d*s)**4

In [33]:
w_sol = sol_w_other(df_1,[df_1["QD"]["profit_s"]-df_1["WH"]["profit_s"]])
w_sol[0][0]

-sqrt(2)*(-alpha + beta*c + beta*delta_d*s)/beta + (-alpha + 2*beta*c + beta*delta_d*s)/beta

In [34]:
display_solve(df_1,w_sol[0][0])

w


-(-sqrt(2)*alpha + alpha - 2*beta*c + sqrt(2)*beta*c - beta*delta_d*s + sqrt(2)*beta*delta_d*s)/beta


epsilon


sqrt(2)*(-4 + 3*sqrt(2))/(2*beta)


T


...

-beta*(-1 + 2*sqrt(2))/(4*(-2 + sqrt(2))**2)


h1_s_e


-beta*(-alpha + beta*c + beta*delta_d*s)**2/(8*(-2 + sqrt(2))**2)


h2_s


-beta**2*(-alpha + beta*c + beta*delta_d*s)**2/(64*(-2 + sqrt(2))**2)

In [35]:
w_sol = sol_w_other(df_1,[df_1["QD"]["profit_all"]-df_1["FB"]["profit_all"]])
w_sol[0][0]

-(-alpha + beta*delta_d*s)/beta

In [36]:
display_solve(df_1,w_sol[0][0])

w


-(-alpha + beta*delta_d*s)/beta


epsilon


1/beta


T


...

4*zoo*(-alpha + beta*c + beta*delta_d*s)**2


h1_s_e


zoo*(-alpha + beta*c + beta*delta_d*s)**4


h2_s


zoo*(-alpha + beta*c + beta*delta_d*s)**4

# BS

In [38]:
df_1 = BS_all

In [39]:
w_sol = sol_w_other(df_1,[df_1["QD"]["profit_r"]])
w_sol[0][0]

(2*alpha*delta_d - alpha*delta_s - alpha + beta*delta_d*s - beta*delta_s*s)/(beta*delta_s - beta)

In [41]:
display_solve(df_1,w_sol[0][0])

w


(2*alpha*delta_d - alpha*delta_s - alpha + beta*delta_d*s - beta*delta_s*s)/(beta*(delta_s - 1))


epsilon


(2*delta_d - delta_s - 1)/(beta*(delta_s - 1))


T


...

zoo*(delta_s - 1)**4*(2*alpha*delta_d - alpha*delta_s - alpha - beta*c*delta_s + beta*c + beta*delta_d*s - beta*delta_s*s)**2/(2*(2*delta_d - delta_s - 1))


h1_s_e


zoo*(delta_s - 1)**4*(2*alpha*delta_d - alpha*delta_s - alpha - beta*c*delta_s + beta*c + beta*delta_d*s - beta*delta_s*s)**4/(2*(2*delta_d - delta_s - 1)**3)


h2_s


-zoo*(delta_s - 1)**6*(2*alpha*delta_d - alpha*delta_s - alpha - beta*c*delta_s + beta*c + beta*delta_d*s - beta*delta_s*s)**4/(16*(2*delta_d - delta_s - 1)**4)

In [47]:
w_sol = sol_w_other(df_1,[df_1["QD"]["profit_s"]-df_1["WH"]["profit_s"]])
w_sol[0][0]

-sqrt(2)*(delta_s - 1)*(2*alpha*delta_d - alpha*delta_s - alpha - beta*c*delta_s + beta*c + beta*delta_d*s - beta*delta_s*s)/(beta*(delta_s**2 - 2*delta_s + 1)) - (2*alpha*delta_d - alpha*delta_s - alpha - 2*beta*c*delta_s + 2*beta*c + beta*delta_d*s - beta*delta_s*s)/(beta*(delta_s - 1))

In [43]:
display_solve(df_1,w_sol[0][0])

w


-(2*alpha*delta_d + 2*sqrt(2)*alpha*delta_d - sqrt(2)*alpha*delta_s - alpha*delta_s - sqrt(2)*alpha - alpha - 2*beta*c*delta_s - sqrt(2)*beta*c*delta_s + sqrt(2)*beta*c + 2*beta*c + beta*delta_d*s + sqrt(2)*beta*delta_d*s - sqrt(2)*beta*delta_s*s - beta*delta_s*s)/(beta*(delta_s - 1))


epsilon


sqrt(2)*(4 + 3*sqrt(2))*(2*delta_d - delta_s - 1)/(2*beta*(delta_s - 1))


T


...

-sqrt(2)*beta*(sqrt(2) + 4)*(delta_s - 1)**2/(8*(sqrt(2) + 2)**2*(2*delta_d - delta_s - 1))


h1_s_e


beta*(delta_s - 1)**2*(2*alpha*delta_d - alpha*delta_s - alpha - beta*c*delta_s + beta*c + beta*delta_d*s - beta*delta_s*s)**2/(8*(sqrt(2) + 2)**2*(2*delta_d - delta_s - 1)**3)


h2_s


-beta**2*(delta_s - 1)**4*(2*alpha*delta_d - alpha*delta_s - alpha - beta*c*delta_s + beta*c + beta*delta_d*s - beta*delta_s*s)**2/(64*(sqrt(2) + 2)**2*(2*delta_d - delta_s - 1)**4)

In [ ]:
display_solve2(df_1,w_sol[0][0])

w


(sqrt(2)*(delta_s - 1)**2*(-2*alpha*delta_d + alpha*delta_s + alpha + beta*c*delta_s - beta*c - beta*delta_d*s + beta*delta_s*s) + (delta_s**2 - 2*delta_s + 1)*(-2*alpha*delta_d + alpha*delta_s + alpha + 2*beta*c*delta_s - 2*beta*c - beta*delta_d*s + beta*delta_s*s))/(beta*(delta_s - 1)*(delta_s**2 - 2*delta_s + 1))


epsilon


(4*sqrt(2)*delta_d + 6*delta_d - 3*delta_s - 2*sqrt(2)*delta_s - 3 - 2*sqrt(2))/(beta*(delta_s - 1))


T


...


h1_all_p


h1_r_p


4*beta*(-2*sqrt(2)*delta_d - 2*delta_d + delta_s + sqrt(2)*delta_s + 1 + sqrt(2))


h1_s_w


beta*(-2*sqrt(2)*delta_s**2 - delta_s**2 + 2*delta_s + 4*sqrt(2)*delta_s - 2*sqrt(2) - 1)/(8*(4*sqrt(2)*delta_d + 6*delta_d - 3*delta_s - 2*sqrt(2)*delta_s - 3 - 2*sqrt(2)))


h1_s_e


h2_s


In [46]:
w_sol = sol_w_other(df_1,[df_1["QD"]["profit_s"]-df_1["FB"]["profit_all"]])
w_sol[0][0]

-2*sqrt((delta_s - 1)**3*(2*delta_d - delta_s - 1))*(-alpha*delta_s + alpha + beta*c*delta_s - beta*c + beta*delta_d*s - beta*delta_s*s)/(beta*(delta_s**3 - 3*delta_s**2 + 3*delta_s - 1)) - (2*alpha*delta_d - alpha*delta_s - alpha - 2*beta*c*delta_s + 2*beta*c + beta*delta_d*s - beta*delta_s*s)/(beta*(delta_s - 1))